The `ipython_blocking` package allows the capture of cell execution, effectively 'blocking' further action until you release.  The original intended use-case is to block until a widget value has changed or other sort of validation of widget input has been done.

In [1]:
import ipython_blocking
import ipywidgets as widgets

The simplest example is blocking further cell execution until a widget value has changed.  Run the next three cells which create a widget, start the 'blocking' context, then print the widget value *only after the widget value has changed to something other than an empty string*.

In [2]:
dd = widgets.Dropdown(options=['', 'foo', 'bar', 'baz'])
dd

Dropdown(options=('', 'foo', 'bar', 'baz'), value='')

In [22]:
ctx = ipython_blocking.CaptureExecution()
with ctx:
    while True:
        if dd.value:
            break
        ctx.step()

In [4]:
print(dd.value)

bar


The explicit context creation above is also wrapped into an IPython line magic called `%block`.  That magic expects one positional argument, which should either be the variable name of a single widget or an arbitrary function.

If the positional argument is a single widget, the `CaptureExecution` context will stop when that widget changes value.  If the positional argument is a function, then it will stop when the called function returns `True`.  Be careful that the function doesn't do anything very complicated since it will be called very often.

The magic command also accepts a single optional `-t` or `--timeout` argument, which is a time in seconds until the context ends on its own.  The default is to not have a timeout.

In [5]:
ipython_blocking.load_ipython_extensions()

The next three lines are nearly identical to the first but will capture cell execution until the `dd2` widget value has changed.

In [6]:
dd2 = widgets.Dropdown(options=['one', 'two', 'three'])
dd2

Dropdown(options=('one', 'two', 'three'), value='one')

In [8]:
%block dd2

In [9]:
print(dd2.value)

three


`%block` can also accept a `ButtonWidget` subclass, and will break when that button is clicked.  It's a good idea to build validation for your form around making the button clickable.  In the example below, the button becomes clickable after the input text is 14 characters or more.  The follow on cells 'block' until the button is clicked.

In [10]:
query_input = widgets.Text(description="Query string:")
button = widgets.Button(description="Submit", disabled=True)

def input_observe(ev):
    value = ev['new']
    if len(value) >= 14:
        button.disabled = False
        button.button_style = 'success'

query_input.observe(input_observe, 'value')
box = widgets.VBox(children=[query_input, button])
box

VBox(children=(Text(value='', description='Query string:'), Button(description='Submit', disabled=True, style=ButtonStyle())))

In [12]:
%block button

In [13]:
print(query_input.value)

foo bar baz foo bar


Next is an example of using the `%block` magic with a more arbitrary validation function.  This will block cell execution until both dropdown widgets are not null and not equal to each other.

In [14]:
dd3 = widgets.Dropdown(options=['', 'foo', 'bar', 'baz'])
dd4 = widgets.Dropdown(options=['', 'foo', 'bar', 'baz'])
box2 = widgets.VBox(children=[dd3, dd4])
box2

VBox(children=(Dropdown(options=('', 'foo', 'bar', 'baz'), value=''), Dropdown(options=('', 'foo', 'bar', 'baz'), value='')))

In [16]:
def baz_validation():
    # return False if dd3 or dd4 is empty, or if dd3 == dd4
    return dd3.value and dd4.value and dd3.value != dd4.value

%block baz_validation

In [17]:
print(dd3.value, dd4.value)

foo bar


The `%block` magic accepts one optional argument, `-t` or `--timeout` which is the number of seconds to block for before it stops capturing cell execution and replays events.  The default is to have no timeout, which means it will block forever.

In [29]:
dd5 = widgets.Dropdown(options=['', 'foo', 'bar', 'baz'])
dd5

Dropdown(options=('', 'foo', 'bar', 'baz'), value='')

In [32]:
%block dd5 -t 10

In [31]:
print(dd5.value)